In [2]:
%pip install TSB-UAD

  Using cached TSB_UAD-0.0.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached arch-7.2.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached tsfresh-0.21.0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached hurst-0.0.5-py3-none-any.whl.metadata (3.6 kB)
  Using cached tslearn-0.6.3-py3-none-any.whl.metadata (14 kB)
  Using cached Cython-3.0.12-cp312-cp312-win_amd64.whl.metadata (3.6 kB)
  Using cached stumpy-1.13.0-py3-none-any.whl.metadata (28 kB)
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-non

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from TSB_UAD.utils.visualisation import plotFig
from TSB_UAD.utils.slidingWindows import find_length
from TSB_UAD.models.feature import Window
from TSB_UAD.models.iforest import IForest
from TSB_UAD.vus.metrics import get_metrics

Matplotlib is building the font cache; this may take a moment.


In [ ]:
public_root = "./data/TSB-UAD-Public"
selected_domains = ['NASA-MSL', 'IOPS', 'Genesis', 'YAHOO']

# For each domain, pick the 1st ts file
series_list, label_list = [], []
for dom in selected_domains:
    dom_path = os.path.join(public_root, dom)
    files = [f for f in os.listdir(dom_path) if f.endswith('.out')]
    chosen = files[0]
    df = (pd.read_csv(os.path.join(dom_path, chosen),
                      header=None)
            .dropna()
            .to_numpy())
    data = df[:, 0].astype(float)
    label = df[:, 1].astype(int)
    series_list.append(data)
    label_list.append(label)

norm1_ts = series_list[0]
norm1_labels = label_list[0]
norm2_ts = np.concatenate([series_list[0],series_list[1]], axis=0)
norm2_labels = np.concatenate([label_list[0],label_list[1]], axis=0)
norm3_ts = np.concatenate([series_list[0],series_list[1],series_list[2]], axis=0)
norm3_labels = np.concatenate([label_list[0],label_list[1],label_list[2]], axis=0)
norm4_ts = np.concatenate(series_list, axis=0)
norm4_labels = np.concatenate(label_list, axis=0)